In [23]:
import os
import math
import numpy as np
import pefile
import pywt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from PIL import Image
from tqdm import tqdm
import multiprocessing as mp

def calculate_entropy(data):
    if not data:
        return 0.0
    counts = Counter(data)
    entropy = 0.0
    total = len(data)
    for count in counts.values():
        p = count / total
        entropy -= p * math.log2(p)
    return entropy

def get_import_features(pe, max_dim=128):
    features = [0] * max_dim
    try:
        if hasattr(pe, 'DIRECTORY_ENTRY_IMPORT'):
            imports = []
            for entry in pe.DIRECTORY_ENTRY_IMPORT:
                for imp in entry.imports:
                    if imp.name:
                        imports.append(imp.name.decode('utf-8', 'ignore'))
            for i, func in enumerate(imports[:max_dim]):
                features[i] = hash(func) % 1000  # Hash simplification
    except:
        pass
    return features

def get_section_features(pe, max_sections=8):
    sections = []
    try:
        for section in pe.sections[:max_sections]:
            sections.extend([
                section.get_entropy(),
                section.SizeOfRawData,
                section.Characteristics
            ])
    except:
        pass
    pad_length = max_sections*3 - len(sections)
    return sections + [0]*pad_length

def get_image_features(file_path):
    try:
        with open(file_path, 'rb') as f:
            data = f.read()
        
        arr = np.frombuffer(data, dtype=np.uint8)
        
        actual_size = int(np.ceil(np.sqrt(len(arr))))
        actual_size = actual_size - (actual_size % 2)  # Ensure even size for wavelet
        
        # Reshape
        arr = arr[:actual_size*actual_size]
        arr = np.pad(arr, (0, actual_size*actual_size - len(arr)))
        img = arr.reshape(actual_size, actual_size).astype(float)
        
        img = (img - img.min()) / (img.max() - img.min() + 1e-8)
        
        coeffs = pywt.wavedec2(img, 'haar', level=3)
        features = []
        
        for coef in coeffs:
        
            if isinstance(coef, tuple):
                for detail_coef in coef:
                    features.extend([
                        np.mean(np.abs(detail_coef)),  # Mean energy
                        np.std(detail_coef),           # Standard deviation
                        np.percentile(detail_coef, 90),# 90th percentile
                        np.sum(detail_coef < 0),       # Number of negative coefficients
                        entropy_measure(detail_coef)    # Entropy of coefficients
                    ])
            else:
                features.extend([
                    np.mean(np.abs(coef)),            # Mean energy
                    np.std(coef),                     # Standard deviation
                    np.percentile(coef, 90),          # 90th percentile
                    np.sum(coef < 0),                 # Number of negative coefficients
                    entropy_measure(coef)             # Entropy of coefficients
                ])
        for coef in coeffs:
            flattened = np.array(coef).flatten()
            features.extend(flattened.tolist())
            
        expected_length = 1024
        if len(features) < expected_length:
            features.extend([0.0] * (expected_length - len(features)))
        
        return features[:expected_length]
    
    except Exception as e:
        # print(e)
        return [0.0] * 1024

def entropy_measure(coeffs):
    coeffs = np.abs(coeffs)
    coeffs = coeffs / (np.sum(coeffs) + 1e-8)
    entropy = -np.sum(coeffs * np.log2(coeffs + 1e-8))
    return entropy


def extract_file_features(file_path):
    try:
        pe = pefile.PE(file_path)
    except:
        pe = None
    
    features = []
    # Entropy
    with open(file_path, 'rb') as f:
        data = f.read()
    features.append(calculate_entropy(data))
    
    # Import features
    features += get_import_features(pe) if pe else [0]*128
    
    # Section features
    features += get_section_features(pe) if pe else [0]*24
    
    # Wavelet features
    features += get_image_features(file_path)
    
    return np.array(features, dtype=np.float32)

def process_file(args):
    file_path, label = args
    features = extract_file_features(file_path)
    return features, label

class MalwareClassifier(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.net(x)




def scan(model_path='model.pt', scan_dir='scan_files'):
    device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
    
    checkpoint = torch.load(model_path, map_location=device)
    model = MalwareClassifier(len(checkpoint['mean'])).to(device)
    model.load_state_dict(checkpoint['model'])
    model.eval()
    
    # move to CPU
    mean_np = checkpoint['mean'].cpu().numpy()
    std_np = checkpoint['std'].cpu().numpy()
    
    total = 0
    detected = 0
    for root, _, files in os.walk(scan_dir):
        for f in files:
            file_path = os.path.join(root, f)
            features = extract_file_features(file_path)
            
            features = (features - mean_np) / std_np
            features_tensor = torch.FloatTensor(features).unsqueeze(0).to(device)
            
            with torch.no_grad():
                prob = model(features_tensor).item()
            
            result = 'Malicious' if prob >= 0.75 else 'Benign'
            print(f'{f}: {result} ({prob:.4f})')
            
            total += 1
            if prob >= 0.5:
                detected += 1
    
    if total > 0:
        print(f'\nDetection Rate: {detected/total:.2%} ({detected}/{total})')
    else:
        print('No files found.')



In [24]:
scan()


.DS_Store: Benign (0.0539)
1l00o86h57u2.sys: Benign (0.0122)
0ktWo9DdkxVW.sys: Benign (0.0166)

Detection Rate: 0.00% (0/3)
